In [1]:
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import random
import time
import cvxpy as cp
import pandas as pd
from sklearn.model_selection import train_test_split

/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix


In [118]:
TRAIN_DATA = pd.read_csv('studentspen-train.csv')

In [119]:
TRAIN_DATA

,x3,y3,x4,y4,x5,y5,x6,y6,Digit
0,10,75,0,24,48,0,100,19,0
1,16,69,0,27,40,0,93,15,0
2,11,5,63,0,100,43,89,99,0
3,47,0,86,25,100,66,76,100,0
4,57,0,100,22,95,75,54,100,0
...,...,...,...,...,...,...,...,...,...
3742,0,92,23,66,90,74,100,53,9
3743,43,65,100,84,95,79,92,38,9
3744,25,76,79,68,100,62,97,23,9
3745,45,90,44,65,100,66,92,35,9


In [120]:
TRAIN_DATA_W_LABELS = TRAIN_DATA[(TRAIN_DATA['Digit']==1) | (TRAIN_DATA['Digit']==9)]

In [121]:
TRAIN_DATA_W_LABELS

,x3,y3,x4,y4,x5,y5,x6,y6,Digit
384,0,88,8,70,15,53,15,35,1
385,75,71,58,57,58,42,33,28,1
386,0,85,0,68,13,51,20,34,1
387,80,76,65,61,50,45,40,30,1
388,57,75,36,60,14,44,0,29,1
...,...,...,...,...,...,...,...,...,...
3742,0,92,23,66,90,74,100,53,9
3743,43,65,100,84,95,79,92,38,9
3744,25,76,79,68,100,62,97,23,9
3745,45,90,44,65,100,66,92,35,9


In [6]:
D = 8 # feature dimension

In [84]:
ONLY_LABELS = TRAIN_DATA_W_LABELS[['Digit']]
ONLY_LABELS

,Digit
384,1
385,1
386,1
387,1
388,1
...,...
3742,9
3743,9
3744,9
3745,9


In [8]:
ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 9, 'Digit'] = -1
ONLY_LABELS

/tmp/ipykernel_832056/1048461531.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 9, 'Digit'] = -1


,Digit
384,1
385,1
386,1
387,1
388,1
...,...
3742,-1
3743,-1
3744,-1
3745,-1


In [9]:
TRAIN_DATA_NO_LABEL = TRAIN_DATA_W_LABELS.drop('Digit', axis=1)

In [10]:
TRAIN_DATA_NO_LABEL

,x3,y3,x4,y4,x5,y5,x6,y6
384,0,88,8,70,15,53,15,35
385,75,71,58,57,58,42,33,28
386,0,85,0,68,13,51,20,34
387,80,76,65,61,50,45,40,30
388,57,75,36,60,14,44,0,29
...,...,...,...,...,...,...,...,...
3742,0,92,23,66,90,74,100,53
3743,43,65,100,84,95,79,92,38
3744,25,76,79,68,100,62,97,23
3745,45,90,44,65,100,66,92,35


In [11]:
# X_train, X_validate, y_train, y_validate = train_test_split( 
#     TRAIN_DATA_NO_LABEL, ONLY_LABELS, test_size=0.2, shuffle=True)

In [12]:
X_train=TRAIN_DATA_NO_LABEL.to_numpy()

In [13]:
y_train=ONLY_LABELS.to_numpy()

In [14]:
y_validate

NameError: name 'y_validate' is not defined

In [15]:
X_validate

NameError: name 'X_validate' is not defined

In [16]:
X_train_np = X_train
X_train_np.shape

(747, 8)

In [17]:
y_train_np = y_train
y_train_np.shape

(747, 1)

In [18]:
train_data_qty = len(X_train_np)

In [19]:
C = 2 

In [20]:
w = cp.Variable((D, 1))

In [21]:
b = cp.Variable()

In [22]:
epsilon = cp.Variable((train_data_qty, 1))

In [23]:
objective = cp.Minimize(cp.sum(cp.square(w))*0.5 + cp.sum(cp.square(epsilon)*C))

In [24]:
constraints = [cp.multiply(y_train_np, (X_train_np @ w + b)) >= 1 - epsilon,  epsilon >= 0]

In [25]:
prob = cp.Problem(objective, constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var w = {}, b = {}".format(w.value, b.value))

status: optimal
optimal value 491.5830709514776
optimal var w = [[ 0.00282481]
 [ 0.02713111]
 [ 0.01992298]
 [ 0.00748893]
 [ 0.00798928]
 [-0.02384728]
 [-0.02514261]
 [ 0.01460923]], b = -2.308137420124083


In [41]:
eps = 1e-2 # select support vectors
random.seed(1) # For reproducibility

In [42]:
sup = y_train_np*(X_train_np.dot(w.value)+b.value)-1 #0.01

In [43]:
len(sup)

747

In [44]:
sup_v1 = ((-eps<sup) & (sup<eps)).flatten()

In [45]:
len(sup_v1)

747

In [46]:
import collections

collections.Counter(sup_v1)

Counter({False: 736, True: 11})

In [122]:
print(sup_v1)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

In [123]:
df=pd.DataFrame(sup_v1)

In [124]:
TRAIN_DATA_W_LABELS.reset_index(drop=True, inplace=True)


In [125]:
TRAIN_DATA_W_LABELS

,x3,y3,x4,y4,x5,y5,x6,y6,Digit
0,0,88,8,70,15,53,15,35,1
1,75,71,58,57,58,42,33,28,1
2,0,85,0,68,13,51,20,34,1
3,80,76,65,61,50,45,40,30,1
4,57,75,36,60,14,44,0,29,1
...,...,...,...,...,...,...,...,...,...
742,0,92,23,66,90,74,100,53,9
743,43,65,100,84,95,79,92,38,9
744,25,76,79,68,100,62,97,23,9
745,45,90,44,65,100,66,92,35,9


In [126]:
df.reset_index(drop=True, inplace=True)

In [127]:
df.head

<bound method NDFrame.head of          0
0    False
1    False
2    False
3    False
4    False
..     ...
742  False
743  False
744  False
745  False
746  False

[747 rows x 1 columns]>

In [128]:
TRAIN_DATA_W_LABELS['sup_vec_bool']=df

/tmp/ipykernel_832056/324631880.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TRAIN_DATA_W_LABELS['sup_vec_bool']=df


In [129]:
TRAIN_DATA_W_LABELS.head

<bound method NDFrame.head of      x3  y3   x4  y4   x5  y5   x6  y6  Digit  sup_vec_bool
0     0  88    8  70   15  53   15  35      1         False
1    75  71   58  57   58  42   33  28      1         False
2     0  85    0  68   13  51   20  34      1         False
3    80  76   65  61   50  45   40  30      1         False
4    57  75   36  60   14  44    0  29      1         False
..   ..  ..  ...  ..  ...  ..  ...  ..    ...           ...
742   0  92   23  66   90  74  100  53      9         False
743  43  65  100  84   95  79   92  38      9         False
744  25  76   79  68  100  62   97  23      9         False
745  45  90   44  65  100  66   92  35      9         False
746   0  85   13  56   84  67  100  68      9         False

[747 rows x 10 columns]>

In [130]:
print(TRAIN_DATA_W_LABELS)

     x3  y3   x4  y4   x5  y5   x6  y6  Digit  sup_vec_bool
0     0  88    8  70   15  53   15  35      1         False
1    75  71   58  57   58  42   33  28      1         False
2     0  85    0  68   13  51   20  34      1         False
3    80  76   65  61   50  45   40  30      1         False
4    57  75   36  60   14  44    0  29      1         False
..   ..  ..  ...  ..  ...  ..  ...  ..    ...           ...
742   0  92   23  66   90  74  100  53      9         False
743  43  65  100  84   95  79   92  38      9         False
744  25  76   79  68  100  62   97  23      9         False
745  45  90   44  65  100  66   92  35      9         False
746   0  85   13  56   84  67  100  68      9         False

[747 rows x 10 columns]


In [131]:
TRAIN_DATA_W_LABELS.loc[TRAIN_DATA_W_LABELS['Digit'] == 9, 'Digit'] = -1

In [156]:
sup_vec_df=TRAIN_DATA_W_LABELS[TRAIN_DATA_W_LABELS['sup_vec_bool']==True]

In [157]:
sup_vec_df

,x3,y3,x4,y4,x5,y5,x6,y6,Digit,sup_vec_bool
253,66,82,100,100,77,76,55,51,1,True
264,55,84,100,100,73,75,55,50,1,True
279,59,76,100,100,67,77,44,51,1,True
438,0,83,52,100,98,80,92,49,-1,True
483,78,98,23,100,32,70,99,77,-1,True
489,62,100,43,78,91,70,100,30,-1,True
560,94,98,42,79,96,79,100,40,-1,True
581,39,80,64,95,78,58,100,22,-1,True
590,65,100,3,84,0,52,70,49,-1,True
680,0,87,58,100,87,67,100,31,-1,True


In [137]:
sup_vec_bool=sup_vec_df['Digit']
sup_vec_bool_np=sup_vec_bool.values
sup_vec_bool_np

array([ 1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1])

In [158]:
sup_vec_df=sup_vec_df.drop('sup_vec_bool', axis=1)
sup_vec_df=sup_vec_df.drop('Digit', axis=1)
sup_vec_np2=sup_vec_df.values
sup_vec_np2

array([[ 66,  82, 100, 100,  77,  76,  55,  51],
       [ 55,  84, 100, 100,  73,  75,  55,  50],
       [ 59,  76, 100, 100,  67,  77,  44,  51],
       [  0,  83,  52, 100,  98,  80,  92,  49],
       [ 78,  98,  23, 100,  32,  70,  99,  77],
       [ 62, 100,  43,  78,  91,  70, 100,  30],
       [ 94,  98,  42,  79,  96,  79, 100,  40],
       [ 39,  80,  64,  95,  78,  58, 100,  22],
       [ 65, 100,   3,  84,   0,  52,  70,  49],
       [  0,  87,  58, 100,  87,  67, 100,  31],
       [ 33,  86,  46,  65, 100,  73,  84,  37]])

In [ ]:
sup_vec_np = sup_vec_df.values
sup_vec_np

array([[ 98,  64,  69, 100,  58,  69, 100,  61],
       [ 71,  91, 100, 100,  82,  74,  68,  48],
       [ 66,  82, 100, 100,  77,  76,  55,  51],
       [ 96, 100, 100,  84,  94,  60,  88,  35],
       [100, 100,  89,  75,  79,  49,  76,  24]])

In [ ]:
sup_vecs_from_1vs9 = [[ 98,  64,  69, 100,  58,  69, 100,  61],
       [ 71,  91, 100, 100,  82,  74,  68,  48],
       [ 66,  82, 100, 100,  77,  76,  55,  51],
       [ 96, 100, 100,  84,  94,  60,  88,  35],
       [100, 100,  89,  75,  79,  49,  76,  24]]

In [ ]:
sup_vecs_from_1vs9

[[98, 64, 69, 100, 58, 69, 100, 61],
 [71, 91, 100, 100, 82, 74, 68, 48],
 [66, 82, 100, 100, 77, 76, 55, 51],
 [96, 100, 100, 84, 94, 60, 88, 35],
 [100, 100, 89, 75, 79, 49, 76, 24]]

In [ ]:
y_train['sup_vec_bool'] = sup_v1
labels_for_sup_vec_df = y_train[y_train['sup_vec_bool']==True]
labels_for_sup_vec_df = labels_for_sup_vec_df.drop('sup_vec_bool', axis=1)
labels_for_sup_vec_np = labels_for_sup_vec_df.values
labels_for_sup_vec_np

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array([[-1],
       [ 1],
       [ 1],
       [ 1],
       [ 1]])

In [ ]:
labels_for_sup_vec_1vs9 = [[-1], [ 1], [ 1], [ 1], [ 1]]

In [ ]:
[[0.00330738], [0.0311968], [ 0.02214532], [ 0.01202985], [ 0.00929693], [-0.02798008], [-0.02736712], [ 0.01508687]]
